# Si446x Device Direct Access Byte File (Dblk and Panic)

In [ ]:
from __future__ import print_function
from builtins import *                  # python3 types
from time import sleep
from datetime import datetime
import struct as pystruct
from binascii import hexlify
import os.path

In [ ]:
!pwd
%autosave 0
import sys
sys.path.append("../si446x/si446x")
%run '../si446x/si446x/notebooks/si446x_Device_Layer.ipynb'

In [ ]:
import sys
sys.path.append("../tagnet/tagnet")
from tagmessages import TagMessage, TagPoll, TagGet, TagPut, TagDelete, TagHead
from tagnames import TagName
from tagtlv import TagTlv, TagTlvList, tlv_types

In [ ]:
import datetime
print('Test Start Time: {}'.format(datetime.datetime.now()))
print('Si446x Radio Device Driver Version: {}'.format(si446x_device_version()))

In [ ]:
sys.path.append("../tagfuse/tagfuse")
from Si446xUtils import si446x_device_enable
from Si446xFile import file_get_bytes, file_put_bytes, file_update_attrs, dblk_put_note
from Si446xImage import im_get_dir, im_put_file, im_delete_file

##  Start up and Radio

In [ ]:
radio = si446x_device_enable()

## File Data Verification

In [ ]:
# default paramters
MAX_WAIT            = 10
MAX_RECV            = 255
MAX_PAYLOAD         = 254
MAX_RETRIES         = 10
RADIO_POWER         = 100
SHORT_DELAY         = 0
from datetime import datetime

In [ ]:
BLOCK_SIZE = 512
def verify_file(radio, check_fname, dev, unit, skip=0):
    with open(check_fname, 'rb') as fd:
        start  = datetime.now()
        eof    = False
        offset  = 0
        while not eof:
            fd.seek(offset+skip)
            dbuf = bytearray(fd.read(BLOCK_SIZE))
            sbuf, eof = file_get_bytes(radio, dev, unit, BLOCK_SIZE, offset)
            if sbuf:
                for ix in range(len(sbuf)):
                    if sbuf[ix] != dbuf[ix]:
                        print(hexlify(sbuf))
                        print(hexlify(dbuf))
                        print("\r{} mismatch: offset:{}, {}/{}".format(datetime.now() - start,
                                                            offset+ix,
                                                            hexlify(sbuf[ix-4:ix+4]),
                                                            hexlify(dbuf[ix-4:ix+4])))
                        break
                offset += len(sbuf)
            elif not eof:
                print("\r{} {}".format(datetime.now() - start, 'timeout'), end="")
            sleep(0)
            print("\r{} {}".format(datetime.now() - start, offset), end="")
    print("\ntime: {}, offset: {}".format(datetime.now() - start, offset))

In [ ]:
STOP

## Verify Dblk File

In [ ]:
verify_file(radio, '/home/pi/o/mm/dblk.data', 'dblk', '0')

## Verify Panic File

In [ ]:
verify_file(radio, '/home/pi/o/mm/panic.data', 'panic', '2', skip=512)

## Put Note

In [ ]:
dblk_put_note(radio, 'hello')

## Update File Status

In [ ]:
file_update_attrs(radio, ['<node_id:ffffffffffff>', 'tag', 'sd', '0', 'panic', 'byte', 1], {})

In [ ]:
file_update_attrs(radio, ['<node_id:ffffffffffff>', 'tag', 'sd', '0', 'dblk', 'byte', '0'], {})

In [ ]:
file_update_attrs(radio, ['<node_id:ffffffffffff>', 'tag', 'sd', '0', 'dblk', 'note'], {})

## Get Image Directory

In [ ]:
print(im_get_dir(radio, ['image']))

## Extra

In [ ]:
STOP

In [ ]:
buf, eof = file_get_bytes(radio, 'dblk', 0, 512, 512*4)

In [ ]:
print(len(buf), eof)

In [ ]:
print(hexlify(buf))

In [ ]:
buf, eof = file_get_bytes(radio, 'panic', 0, 1, 76799)

In [ ]:
print(len(buf), eof)

In [ ]:
print(hexlify(buf))

## Get Chip Status

In [ ]:
print(radio.get_chip_status())

## Get Image Directory

In [ ]:
#<node_id>   "tag"  "sd"  0  "img"
image_manager_name = TagName (TagTlv(tlv_types.NODE_ID, -1),
                              'tag',
                              'sd',
                              TagTlv(0),
                              TagTlv('img'))
dir_info = TagGet(image_manager_name)
#print(dir_info.name)
dir_msg = dir_info.build()
#print(len(dir_msg),hexlify(dir_msg))
si446x_device_send_msg(radio, dir_msg, RADIO_POWER);
rsp_buf, rssi, status = si446x_device_receive_msg(radio, MAX_RECV, MAX_WAIT)
if (rsp_buf):
#    print(len(rsp_buf),hexlify(rsp_buf))
    rsp_obj = TagMessage(rsp_buf)
    for x in range(0, 8, 2):
        print("state: {}, {}".format(rsp_obj.payload[x+1].value(), rsp_obj.payload[x]))
else:
    print('timeout')

In [ ]:
#"tag"	"sys"	<node_id>	"which"
def get_version(which):
    sys_name = TagName (TagTlv(tlv_types.NODE_ID, -1),
                        'tag',
                        'sys',
                        TagTlv(which))
    sys_obj = TagGet(sys_name)
#    print(sys_obj.name)
    get_msg = sys_obj.build()
    si446x_device_send_msg(radio, get_msg, RADIO_POWER);
    rsp_buf, rssi, status = si446x_device_receive_msg(radio, MAX_RECV, 5)
    if(rsp_buf):
#        print(hexlify(rsp_buf))
        rsp_obj = TagMessage(rsp_buf)
        print("{}: {:^10} state: {}, {}".format(rsp_obj.header.options.param.error_code, which, rsp_obj.payload[1].value(), rsp_obj.payload[0]))

In [ ]:
get_version('active')
get_version('backup')
get_version('golden')
get_version('nib')
get_version('running')

In [ ]:
#"tag"	"sys"	<node_id>	"which"
def set_version(which, version):
    set_name = TagName (TagTlv(tlv_types.NODE_ID, -1),
                        'tag',
                        'sys',
                        TagTlv(which),
                        TagTlv(tlv_types.VERSION, version))
    set_obj = TagPut(set_name)
#    print(set_obj.name)
    set_msg = set_obj.build()
    si446x_device_send_msg(radio, set_msg, RADIO_POWER);
    rsp_buf, rssi, status = si446x_device_receive_msg(radio, MAX_RECV, 5)
    if(rsp_buf):
#        print(hexlify(rsp_buf))
        rsp_obj = TagMessage(rsp_buf)
#        print(rsp_obj.header)
        if (rsp_obj.payload):
            print("{}: state: {}, {}".format(rsp_obj.header.options.param.error_code, rsp_obj.payload[1].value(), rsp_obj.payload[0]))
        else:
            print("{}".format(rsp_obj.header.options.param.error_code))

In [ ]:
set_version('active', (118, 16, 0))

In [ ]:
set_version('backup', (32, 16, 0))

In [ ]:
set_version('running', (125, 1, 0))

## Load Image File onto Tag

In [ ]:
radio.trace._enable()
# set rssi control (enable latch, average4, latch on sync)
prp         = bytearray('0x22')
radio.set_property('MODEM', 0x4c, prp) 
# tx/rx threshold
#prp = bytearray('\x20\x20')
#radio.set_property('PKT', 0x0b, prp)
start = datetime.now()
print(start)
class RadioLoadException(Exception):
    pass
try:
    # open input file and determine its length
    infile = open(filename, 'rb')
    infile.seek(0, 2) # seek to the end
    file_size = infile.tell()
    if file_size < IMAGE_MIN_SIZE: raise RadioLoadException("input file too short")
    infile.seek(0, 0)    # seek to the beginnnig
    # get image info from input file and sanity check
    infile.seek(IMAGE_META_OFFSET) # seek to location of image info
    image_info = image_info_struct.unpack(infile.read(image_info_struct.size))
    print("file information")
    sig, image_start, imagelength, vector_chk, image_chk, im_build, im_minor, im_major,\
        main_tree, aux_tree, build_time, im_rev, im_model = image_info
    pstr = "  signature: 0x{:x}, start: 0x{:x}, length: 0x{:x}, vect_chk: 0x{:x}, image_chk: 0x{:x}"
    print(pstr.format(sig, image_start, imagelength, vector_chk, image_chk))
    pstr = "  version: ({}.{}.{}(0x{:x})), rev: {}, model: {}"
    print(pstr.format(im_major, im_minor, im_build, im_build, im_rev, im_model))
    if sig != IMAGE_INFO_SIG: raise RadioLoadException("image metadata is invalid")
    infile.seek(0)    # seek to the beginnnig
    # loop to transfer image data to tag
    retries         = 0
    total_retries   = 0
    packets_sent    = 0
    rssi            = 0
    offset          = 0
    path_list = ['<node_id:ffffffffffff>','tag','sd', 0, 'img', '<version:{}.{}.{}>'.format(im_build, im_minor, im_major)]
    while (file_size - infile.tell() > 0):
        if ((packets_sent % 10) == 0):
            print("\r{0:.2f}%".format((float(infile.tell())/float(file_size))*100), end='')
#        print(">>>> file size: {}, offset: {}, retries: {}, rssi: {}\r".format(file_size, infile.tell(), retries, rssi, end=''))
        error, offset = im_put_file(radio, path_list, infile, offset)
        if (error is tlv_errors.SUCCESS):
            infile.seek(offset)
    
        if offset != infile.tell():
            if (rsp_obj is None): p = "TIMEOUT"
            else:                 p = "BAD OFFSET"
            print("\n{}  {}! offset: response {}, expected {}".format(datetime.now(), p, offset, infile.tell()))
            if (offset >= 0):
                infile.seek(offset, 0)
            else:
                infile.seek(-plen, 1)
                offset -= plen
                
            retries = retries + 1
            total_retries = total_retries + 1
        else:
            retries = 0
        if (retries > MAX_RETRIES):
            raise RadioLoadException("too many retries")
        sleep(SHORT_DELAY)
    # send end of file to complete the image load operation
    im_send_request(infile, file_size, (im_build, im_minor, im_major), True)
    rsp_obj, rsp_msg, offset, rssi = im_get_response(infile)
    print("\ntotals bytes: {}, packets: {} retries: {}".format(offset, packets_sent, total_retries))
except RadioLoadException:
    print('terminating')
finally:
    infile.close()
print('\ndone, elapsed time', datetime.now() - start)